

# **Data Science, 2024**



A demonstration of exploratory data analysis to accompany the Lecture.

In [5]:
import numpy as np
import pandas as pd

These options are used to customize the display of pandas DataFrames for better readability and precision when working with data.

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
# This option stops scientific notation for pandas
pd.set_option('display.float_format', '{:.2f}'.format)


# Tuberculosis in the United States

What can we say about the presence of Tuberculosis in the United States?

Let's look at the data included in the [original CDC article](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down) published in 2021.

**CSV and Nice Field Names**

Suppose Table 1 was saved as a CSV file located in `data/cdc_tuberculosis.csv`. We can then explore the CSV (which is a text
file, and does not contain binary-encoded data) in many ways:
1. Using a text editor like the one in DataHub (right-click on the file and use `Open->Editor`), emacs, vim, VSCode, etc.
2. Opening the CSV directly in DataHub (read-only), Excel, Google Sheets, etc.
3. The Python file object
4. pandas, using `pd.read_csv()`

Let's start with the first two so we solidify the idea of a CSV as **rectangular data (i.e., tabular data) stored as comma-separated values**.

Try 1, 2.

Then Let's see the tried-and-true Data Science approach: pandas.

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# Read tuberculosis data
tb=pd.read_csv('sets/pandas 7/cdc_tuberculosis.csv')
tb

NameError: name 'pd' is not defined

Wait, what's up with the "Unnamed" column names? And the first row, for that matter?

Congratulations -- you're ready to wrangle your data. Because of how things are stored, we'll need to clean the data a bit to name our columns better.

A reasonable first step is to identify the row with the right header. The `pd.read_csv()` function ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) has the convenient `header` parameter:

In [3]:
# Answer Here
tb=pd.read_csv('sets/pandas 7/cdc_tuberculosis.csv',header=1)
tb

NameError: name 'pd' is not defined

Wait...but now we can't differentiate betwen the "Number of TB cases" and "TB incidence" year columns. pandas has tried to make our lives easier by automatically adding ".1" to the latter columns, but this doesn't help us as humans understand the data.

We can do this manually with `df.rename()`

In [4]:
# Rename column List
tb=tb.rename(columns={'2019':'TB cases 2019','2020':'TB cases 2020','2021':'TB cases 2021','2019.1':'TB incidence 2019',
                      '2020.1':'TB incidence 2020','2021.1':'TB incidence 2021'})
tb

NameError: name 'tb' is not defined

# Record Granularity

You might already be wondering: What's up with that first record?

Row 0 is what we call a **rollup record**, or summary record. It's often useful when displaying tables to humans. The **granularity** of record 0 (Totals) vs the rest of the records (States) is different.


Okay, EDA step two. How was the rollup record aggregated?

Let's check if Total TB cases is the sum of all state TB cases. If we sum over all rows, we should get **2x** the total cases in each of our TB cases by year (why?).

In [80]:
# Apply sum to tb DataFrame
tb_sum=tb.sum()
tb_sum

U.S. jurisdiction    TotalAlabamaAlaskaArizonaArkansasCaliforniaCol...
TB cases 2019        8,9008758183642,111666718245583029973261085237...
TB cases 2020        7,1737258136591,706525417194122219282169239376...
TB cases 2021        7,8609258129691,750585443194992281064255127494...
TB incidence 2019                                               109.94
TB incidence 2020                                                93.09
TB incidence 2021                                               102.94
dtype: object


Whoa, what's going on? Check out the column types:

In [81]:
# Find datatype for each column
tb[:].dtypes

U.S. jurisdiction     object
TB cases 2019         object
TB cases 2020         object
TB cases 2021         object
TB incidence 2019    float64
TB incidence 2020    float64
TB incidence 2021    float64
dtype: object

Looks like those commas are causing all TB cases to be read as the `object` datatype, or **storage type** (close to the Python string datatype), so pandas is concatenating strings instead of adding integers.

Fortunately `read_csv` also has a `thousands` parameter (for what it's worth, I didn't know this beforehand--I [googled](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) this):

In [84]:
tb=pd.read_csv('sets/pandas 7/cdc_tuberculosis.csv',header=1,thousands=",").rename(columns={'2019':'TB cases 2019','2020':'TB cases 2020','2021':'TB cases 2021','2019.1':'TB incidence 2019',
                      '2020.1':'TB incidence 2020','2021.1':'TB incidence 2021'})
tb

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


In [64]:
# now apply sum
# Answer Here
tb_sum=tb.sum()
tb_sum

U.S. jurisdiction    TotalAlabamaAlaskaArizonaArkansasCaliforniaCol...
2019                                                             17800
2020                                                             14346
2021                                                             15720
2019.1                                                          109.94
2020.1                                                           93.09
2021.1                                                          102.94
dtype: object

The Total TB cases look right. Phew!

(We'll leave it to your own EDA to figure out how the TB incidence "Totals" were aggregated.)

Let's just look at the records with **state-level granularity**:

In [85]:
# Answer Here
tb_state=tb.iloc[1:,:]
tb_state

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
5,California,2111,1706,1750,5.35,4.32,4.46
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


What do each of these values represent? Why?

To the lecture!


# Gather Census Data

U.S. Census population estimates [source](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html) (2019), [source](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html) (2020-2021).

Running the below cells cleans the data. We encourage you to closely explore the CSV and study these lines after lecture...

There are a few new methods here:
* `df.convert_dtypes()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.convert_dtypes.html)) conveniently converts all float dtypes into ints and is out of scope for the class.
* `df.drop_na()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)) will be explained in more detail next time.

In [204]:
# Load 2010s census data
# Answer
census19=pd.read_csv('sets/pandas 7/nst-est2019-01.csv',header=3,thousands=",")
census19

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538.00,308758105.00,309321666.00,311556874.00,313830990.00,315993715.00,318301008.00,320635163.00,322941311.00,324985539.00,326687501.00,328239523.00
1,Northeast,55317240.00,55318443.00,55380134.00,55604223.00,55775216.00,55901806.00,56006011.00,56034684.00,56042330.00,56059240.00,56046620.00,55982803.00
2,Midwest,66927001.00,66929725.00,66974416.00,67157800.00,67336743.00,67560379.00,67745167.00,67860583.00,67987540.00,68126781.00,68236628.00,68329004.00
3,South,114555744.00,114563030.00,114866680.00,116006522.00,117241208.00,118364400.00,119624037.00,120997341.00,122351760.00,123542189.00,124569433.00,125580448.00
4,West,71945553.00,71946907.00,72100436.00,72788329.00,73477823.00,74167130.00,74925793.00,75742555.00,76559681.00,77257329.00,77834820.00,78347268.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Note: The estimates are based on the 2010 Cens...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Table 1. Annual Estimates of the Resident Popu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Apply some EDA

Drop the column of Estiamte Base

Rename Unnamed: 0 as 'Geographic Area'

"smart" converting of columns(.convert_dtypes()) Google it. Use at your own risk

.dropna() to drop records with NaN

You can also suggest any change that can be helpful for EDA



In [205]:
census19=census19.drop(['Estimates Base'],axis=1)
census19

,Unnamed: 0,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538.00,309321666.00,311556874.00,313830990.00,315993715.00,318301008.00,320635163.00,322941311.00,324985539.00,326687501.00,328239523.00
1,Northeast,55317240.00,55380134.00,55604223.00,55775216.00,55901806.00,56006011.00,56034684.00,56042330.00,56059240.00,56046620.00,55982803.00
2,Midwest,66927001.00,66974416.00,67157800.00,67336743.00,67560379.00,67745167.00,67860583.00,67987540.00,68126781.00,68236628.00,68329004.00
3,South,114555744.00,114866680.00,116006522.00,117241208.00,118364400.00,119624037.00,120997341.00,122351760.00,123542189.00,124569433.00,125580448.00
4,West,71945553.00,72100436.00,72788329.00,73477823.00,74167130.00,74925793.00,75742555.00,76559681.00,77257329.00,77834820.00,78347268.00
...,...,...,...,...,...,...,...,...,...,...,...,...
58,Note: The estimates are based on the 2010 Cens...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Table 1. Annual Estimates of the Resident Popu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
census19=census19.rename(columns={'Unnamed: 0':'Geographic Area'})
census19

,Geographic Area,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538.00,309321666.00,311556874.00,313830990.00,315993715.00,318301008.00,320635163.00,322941311.00,324985539.00,326687501.00,328239523.00
1,Northeast,55317240.00,55380134.00,55604223.00,55775216.00,55901806.00,56006011.00,56034684.00,56042330.00,56059240.00,56046620.00,55982803.00
2,Midwest,66927001.00,66974416.00,67157800.00,67336743.00,67560379.00,67745167.00,67860583.00,67987540.00,68126781.00,68236628.00,68329004.00
3,South,114555744.00,114866680.00,116006522.00,117241208.00,118364400.00,119624037.00,120997341.00,122351760.00,123542189.00,124569433.00,125580448.00
4,West,71945553.00,72100436.00,72788329.00,73477823.00,74167130.00,74925793.00,75742555.00,76559681.00,77257329.00,77834820.00,78347268.00
...,...,...,...,...,...,...,...,...,...,...,...,...
58,Note: The estimates are based on the 2010 Cens...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Table 1. Annual Estimates of the Resident Popu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
census19=census19.convert_dtypes()
census19.dtypes

Geographic Area    string
Census              Int64
2010                Int64
2011                Int64
2012                Int64
2013                Int64
2014                Int64
2015                Int64
2016                Int64
2017                Int64
2018                Int64
2019                Int64
dtype: object

In [208]:
census19=census19.dropna()
census19

,Geographic Area,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
1,Northeast,55317240,55380134,55604223,55775216,55901806,56006011,56034684,56042330,56059240,56046620,55982803
2,Midwest,66927001,66974416,67157800,67336743,67560379,67745167,67860583,67987540,68126781,68236628,68329004
3,South,114555744,114866680,116006522,117241208,118364400,119624037,120997341,122351760,123542189,124569433,125580448
4,West,71945553,72100436,72788329,73477823,74167130,74925793,75742555,76559681,77257329,77834820,78347268
...,...,...,...,...,...,...,...,...,...,...,...,...
52,.Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893
53,.West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147
54,.Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434
55,.Wyoming,563626,564487,567299,576305,582122,582531,585613,584215,578931,577601,578759


In [209]:
# census 2020s data
census21=pd.read_csv('sets/pandas 7/NST-EST2022-POP.csv')
census21

,table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN
1,Geographic Area,"April 1, 2020 Estimates Base",Population Estimate (as of July 1),NaN,NaN
2,NaN,NaN,2020,2021,2022
3,United States,"331,449,520","331,511,512","332,031,554","333,287,557"
4,Northeast,"57,609,156","57,448,898","57,259,257","57,040,406"
...,...,...,...,...,...
61,Note: The estimates are developed from a base ...,NaN,NaN,NaN,NaN
62,Suggested Citation:,NaN,NaN,NaN,NaN
63,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN
64,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN


In [210]:
# perform EDA Here as did on 2010s census data
census21=pd.read_csv('sets/pandas 7/NST-EST2022-POP.csv',header=3,thousands=',')
census21=census21.rename(columns={'Unnamed: 0':'Geographic Area'})
census21=census21.drop('Unnamed: 1',axis=1)
census21=census21.dropna()
census21=census21.convert_dtypes()
census21

,Geographic Area,2020,2021,2022
0,United States,331511512,332031554,333287557
1,Northeast,57448898,57259257,57040406
2,Midwest,68961043,68836505,68787595
3,South,126450613,127346029,128716192
4,West,78650958,78589763,78743364
...,...,...,...,...
52,.Washington,7724031,7740745,7785786
53,.West Virginia,1791420,1785526,1775156
54,.Wisconsin,5896271,5880101,5892539
55,.Wyoming,577605,579483,581381




# Join Data (Merge DataFrames)


Time to `merge`! Here I use the DataFrame method `df1.merge(right=df2, ...)` on DataFrame `df1` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)). Contrast this with the function `pd.merge(left=df1, right=df2, ...)` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html?highlight=pandas%20merge#pandas.merge)). Feel free to use either.

In [221]:
#Remove dot
census19['Geographic Area']=census19['Geographic Area'].str.lstrip('.')
census21['Geographic Area']=census21['Geographic Area'].str.lstrip('.')

In [224]:
# merge TB dataframe with two US census dataframes
census=pd.merge(tb,census19,left_on=['U.S. jurisdiction'],right_on=['Geographic Area'],suffixes=['_x','_y'])\
        .merge(census21,on=['Geographic Area'],suffixes=[('_x','_y')])
census

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Alabama,87,72,92,1.77,1.43,1.83,Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,5031362,5049846,5074296
1,Alaska,58,58,58,7.91,7.92,7.92,Alaska,710231,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,732923,734182,733583
2,Arizona,183,136,129,2.51,1.89,1.77,Arizona,6392017,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,7179943,7264877,7359197
3,Arkansas,64,59,69,2.12,1.96,2.28,Arkansas,2915918,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,3014195,3028122,3045637
4,California,2111,1706,1750,5.35,4.32,4.46,California,37253956,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,39501653,39142991,39029342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Virginia,191,169,161,2.23,1.96,1.86,Virginia,8001024,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519,8636471,8657365,8683619
47,Washington,221,163,199,2.90,2.11,2.57,Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893,7724031,7740745,7785786
48,West Virginia,9,13,7,0.50,0.73,0.39,West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147,1791420,1785526,1775156
49,Wisconsin,51,35,66,0.88,0.59,1.12,Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434,5896271,5880101,5892539


This is a little unwieldy. We could either drop the unneeded columns now, or just merge on smaller census DataFrames. Let's do the latter.

In [220]:
# try merging again, but cleaner this time

census21

,Geographic Area,2020,2021,2022
0,United States,331511512,332031554,333287557
1,Northeast,57448898,57259257,57040406
2,Midwest,68961043,68836505,68787595
3,South,126450613,127346029,128716192
4,West,78650958,78589763,78743364
...,...,...,...,...
52,Washington,7724031,7740745,7785786
53,West Virginia,1791420,1785526,1775156
54,Wisconsin,5896271,5880101,5892539
55,Wyoming,577605,579483,581381



## Reproduce incidence

Let's recompute incidence to make sure we know where the original CDC numbers came from.

From the [CDC report](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down): TB incidence is computed as “Cases per 100,000 persons using mid-year population estimates from the U.S. Census Bureau.”

If we define a group as 100,000 people, then we can compute the TB incidence for a given state population as

$$\text{TB incidence} = \frac{\text{# TB cases in population}}{\text{# groups in population}} = \frac{\text{# TB cases in population}}{\text{population}/100000} $$

$$= \frac{\text{# TB cases in population}}{\text{population}} \times 100000$$

Let's try this for 2019:

In [226]:
census["recompute incidence 2019"] = census["TB cases 2019"]/census["2019"]*100000
census

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,recompute incidence 2019
0,Alabama,87,72,92,1.77,1.43,1.83,Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,5031362,5049846,5074296,1.77
1,Alaska,58,58,58,7.91,7.92,7.92,Alaska,710231,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,732923,734182,733583,7.93
2,Arizona,183,136,129,2.51,1.89,1.77,Arizona,6392017,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,7179943,7264877,7359197,2.51
3,Arkansas,64,59,69,2.12,1.96,2.28,Arkansas,2915918,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,3014195,3028122,3045637,2.12
4,California,2111,1706,1750,5.35,4.32,4.46,California,37253956,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,39501653,39142991,39029342,5.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Virginia,191,169,161,2.23,1.96,1.86,Virginia,8001024,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519,8636471,8657365,8683619,2.24
47,Washington,221,163,199,2.90,2.11,2.57,Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893,7724031,7740745,7785786,2.90
48,West Virginia,9,13,7,0.50,0.73,0.39,West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147,1791420,1785526,1775156,0.50
49,Wisconsin,51,35,66,0.88,0.59,1.12,Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434,5896271,5880101,5892539,0.88


Awesome!!!

Let's use a for-loop and Python format strings to compute TB incidence for all years. Python f-strings are just used for the purposes of this demo, but they're handy to know when you explore data beyond this course ([Python documentation](https://docs.python.org/3/tutorial/inputoutput.html)).

In [227]:
# recompute incidence for all years (2019, 2020, 2021)
census["recompute incidence 2020"] = census["TB cases 2020"]/census["2020"]*100000
census["recompute incidence 2021"] = census["TB cases 2021"]/census["2021"]*100000
census

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,recompute incidence 2019,recompute incidence 2020,recompute incidence 2021
0,Alabama,87,72,92,1.77,1.43,1.83,Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,5031362,5049846,5074296,1.77,1.43,1.82
1,Alaska,58,58,58,7.91,7.92,7.92,Alaska,710231,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,732923,734182,733583,7.93,7.91,7.90
2,Arizona,183,136,129,2.51,1.89,1.77,Arizona,6392017,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,7179943,7264877,7359197,2.51,1.89,1.78
3,Arkansas,64,59,69,2.12,1.96,2.28,Arkansas,2915918,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,3014195,3028122,3045637,2.12,1.96,2.28
4,California,2111,1706,1750,5.35,4.32,4.46,California,37253956,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,39501653,39142991,39029342,5.34,4.32,4.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Virginia,191,169,161,2.23,1.96,1.86,Virginia,8001024,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519,8636471,8657365,8683619,2.24,1.96,1.86
47,Washington,221,163,199,2.90,2.11,2.57,Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893,7724031,7740745,7785786,2.90,2.11,2.57
48,West Virginia,9,13,7,0.50,0.73,0.39,West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147,1791420,1785526,1775156,0.50,0.73,0.39
49,Wisconsin,51,35,66,0.88,0.59,1.12,Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434,5896271,5880101,5892539,0.88,0.59,1.12


These numbers look pretty close!!! There are a few errors in the hundredths place, particularly in 2021. It may be useful to further explore reasons behind this discrepancy. We'll leave it to you!

In [20]:
tb_census_df.describe()

,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,2010,2011,2012,2013,...,2016,2017,2018,2019,2020,2021,2022,recompute incidence 2019,recompute incidence 2020,recompute incidence 2021
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.0,51.0,51.0,51.0,...,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0
mean,174.509804,140.647059,154.117647,2.102549,1.782941,1.971961,6065130.705882,6108958.313725,6153548.823529,6195955.196078,...,6332182.568627,6372265.470588,6405637.27451,6436069.078431,6500225.72549,6510422.627451,6535050.137255,2.104969,1.784655,1.969928
std,341.738752,271.055775,286.781007,1.498745,1.337414,1.478468,6838021.594872,6901809.616229,6967072.873326,7029240.826807,...,7235903.896549,7288669.592645,7327257.808679,7360660.467814,7408168.462614,7394300.076705,7423508.194422,1.500236,1.338263,1.474929
min,1.000000,0.000000,2.000000,0.170000,0.000000,0.210000,564487.0,567299.0,576305.0,582122.0,...,584215.0,578931.0,577601.0,578759.0,577605.0,579483.0,581381.0,0.172783,0.0,0.210049
25%,25.500000,29.000000,23.000000,1.295000,1.210000,1.235000,1700144.0,1712291.0,1724313.5,1732560.0,...,1756701.5,1767359.5,1777413.5,1789606.0,1820311.0,1844920.0,1857094.5,1.297485,1.211433,1.233905
50%,70.000000,67.000000,69.000000,1.800000,1.520000,1.700000,4348181.0,4369821.0,4386346.0,4404659.0,...,4438182.0,4452268.0,4461153.0,4467673.0,4507445.0,4506589.0,4512310.0,1.808606,1.521612,1.694502
75%,180.500000,139.000000,150.000000,2.575000,1.990000,2.220000,6654568.5,6720105.0,6780031.5,6838650.0,...,7117921.5,7233685.0,7340946.5,7446805.0,7451987.0,7502811.0,7572491.5,2.577577,1.993607,2.219482
max,2111.000000,1706.000000,1750.000000,7.910000,7.920000,7.920000,37319502.0,37638369.0,37948800.0,38260787.0,...,39167117.0,39358497.0,39461588.0,39512223.0,39501653.0,39142991.0,39029342.0,7.928425,7.913519,7.899949
